# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [47]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.datapath import DataPath

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.56.0


## Dataset

### Overview
In this project, we are forcusing on deploying the model endpoint to predict the Hearth failure, we used the both AutoML and Hyperparameter turning to train the data and find the best model among them. We will use it to deploy the model endpoint and consume it to predict patient's survival based on the collected patient data.

We analyzed a dataset containing the medical records of 299 heart failure patients collected at the Faisalabad Institute of Cardiology and at the Allied Hospital in Faisalabad. The patients consisted of 105 woman and 194 men, and their ages between 40 and 95 year old.

In [40]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-heart-failure'

experiment=Experiment(ws, experiment_name)

In [54]:
found = False
key = "Heart Failure Dataset"
description_text = "Heart Failure Dataset for Udacity Capstone"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['../dataset/heart_failure_clinical_records_dataset.csv'],  target_path='data/', overwrite=True, show_progress=True)

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

if not found:
    example_data = 'data/heart_failure_clinical_records_dataset.csv'
    datastore_path =[
        DataPath(datastore, example_data)
    ]
    dataset = Dataset.Tabular.from_delimited_files(path=datastore_path)
    dataset = dataset.register(workspace=ws,name=key,description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

Uploading an estimated of 1 files
Uploading ../dataset/heart_failure_clinical_records_dataset.csv
Uploaded ../dataset/heart_failure_clinical_records_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,581.84,38.08,263358.03,1.39,136.63,130.26
std,11.89,970.29,11.83,97804.24,1.03,4.41,77.61
min,40.00,23.00,14.00,25100.00,0.50,113.00,4.00
25%,51.00,116.50,30.00,212500.00,0.90,134.00,73.00
50%,60.00,250.00,38.00,262000.00,1.10,137.00,115.00
75%,70.00,582.00,45.00,303500.00,1.40,140.00,203.00
max,95.00,7861.00,80.00,850000.00,9.40,148.00,285.00


In [55]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,False,582,False,20,True,265000.00,1.90,130,True,False,4,True
1,55.00,False,7861,False,38,False,263358.03,1.10,136,True,False,6,True
2,65.00,False,146,False,20,False,162000.00,1.30,129,True,True,7,True
3,50.00,True,111,False,20,False,210000.00,1.90,137,True,False,7,True
4,65.00,True,160,True,20,False,327000.00,2.70,116,False,False,8,True


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [12]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "hyper-parameter-turning"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [13]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task = "classification",
    training_data=dataset,
    label_column_name="DEATH_EVENT",   
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    enable_onnx_compatible_models=True,
    **automl_settings
)

In [14]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure,AutoML_acb8774e-01cb-4071-95a7-ea891be61c9e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure,AutoML_acb8774e-01cb-4071-95a7-ea891be61c9e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

{'runId': 'AutoML_acb8774e-01cb-4071-95a7-ea891be61c9e',
 'target': 'hyper-parameter-turning',
 'status': 'Completed',
 'startTimeUtc': '2024-07-18T02:53:22.730462Z',
 'endTimeUtc': '2024-07-18T03:07:46.434904Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'hyper-parameter-turning',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-heart-failure","subscription_id":"eab0c039-c811-4365-a933-8367a6b19fe6","resource_group":"azureml","workspace_name":"test123","region":"southeastasia","compute_target":"hyper-parameter

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [63]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
print("---------------------------------------------------------------------------")
print(f'Run Id: {best_run.id}')
print(f'Accuracy: {best_run_metrics["accuracy"]}')
print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
Run Id: AutoML_acb8774e-01cb-4071-95a7-ea891be61c9e_39
Accuracy: 0.8829885057471264
---------------------------------------------------------------------------


In [36]:
#TODO: Save the best model
print("the model is saved in the outputs directory")
os.makedirs("./outputs", exist_ok = True)
best_run.download_file("outputs/model.pkl", "outputs/automl_model.pkl")
best_run.download_file("outputs/model.onnx", "outputs/automl_model.onnx")
best_run.download_file("outputs/scoring_file_v_1_0_0.py", "outputs/score.py")
best_run.download_file("outputs/conda_env_v_1_0_0.yml", "outputs/env.yml")
print("successful")

the model is saved in the outputs directory
successful


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.


TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [18]:
automl_run, fitted_model = remote_run.get_output(iteration=2)

In [19]:
original_model = automl_run.register_model(
    model_name="automl_model", model_path="outputs/model.pkl"
)

In [71]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.core.environment import Environment

service_name = 'voting-ensemble-endpoint'
myenv = Environment.from_conda_specification(name="myenv",file_path='outputs/env.yml')

# myenv = get_environment_safe(automl_run)
inference_config = InferenceConfig(entry_script='outputs/score.py', environment=myenv)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1.8, 
    memory_gb=2, 
    enable_app_insights=True, 
    auth_enabled=True
)

service = Model.deploy(
    workspace=ws,
    name=service_name,
    inference_config=inference_config,
    models=[original_model],
    deployment_config=aci_config,
    overwrite=True
)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-07-18 15:42:53+00:00 Creating Container Registry if not exists.
2024-07-18 15:42:54+00:00 Use the existing image.
2024-07-18 15:42:55+00:00 Submitting deployment to compute.
2024-07-18 15:43:03+00:00 Checking the status of deployment voting-ensemble-endpoint..
2024-07-18 15:45:01+00:00 Checking the status of inference endpoint voting-ensemble-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [72]:
print(service.scoring_uri)
print(service.get_keys())

http://d26fdb76-c5cf-475b-954a-709187f044c0.southeastasia.azurecontainer.io/score
('XeZ59SwDjK6IT2wKdsNLRZGhcS6jVMRC', 'zTOupXnlXNRvxolMVuB4j3yf7BrbGxl6')


In [83]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = service.get_keys()[0]

# Two sets of data to score, so we get two results back
data = {
  "data": [
        {
            "age": 70, 
            "anaemia": False, 
            "creatinine_phosphokinase": 92, 
            "diabetes": False, 
            "ejection_fraction": 60, 
            "high_blood_pressure": True, 
            "platelets": 317000, 
            "serum_creatinine": 0.8, 
            "serum_sodium": 140, 
            "sex": False, 
            "smoking": True, 
            "time": 74
        }
    ],
    "method": "predict"
}

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [false]}


TODO: In the cell below, print the logs of the web service and delete the service

In [82]:
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

2024-07-18T15:44:40,648746679+00:00 - rsyslog/run 
2024-07-18T15:44:40,655426342+00:00 - gunicorn/run 
2024-07-18T15:44:40,657604736+00:00 | gunicorn/run | 
2024-07-18T15:44:40,658805789+00:00 - nginx/run 
2024-07-18T15:44:40,660657927+00:00 | gunicorn/run | ###############################################
2024-07-18T15:44:40,662242936+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-07-18T15:44:40,665358452+00:00 | gunicorn/run | ###############################################
2024-07-18T15:44:40,667518385+00:00 | gunicorn/run | 
2024-07-18T15:44:40,670314576+00:00 | gunicorn/run | 
2024-07-18T15:44:40,678636294+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240418.v1
2024-07-18T15:44:40,679552054+00:00 | gunicorn/run | 
2024-07-18T15:44:40,680423340+00:00 | gunicorn/run | 
2024-07-18T15:44:40,681291221+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml_b642e377c6ff7effcfe8950f24786eb4/bin:/opt/mi

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
